**Связь docker контейнера и файловой системы хоста**

# Команда `cp`

Позволяет производить обмен файлами между контейнером и хостом при запущеном контейнере.

### Хост -> контейнер

Синтаксис использования комманды следующий:

`docker cp <путь на хосте> <контейнер в который производится копирование>:<путь в контейнере>`

Далее пример, как я в работающий контейнер закину файл "my_datafile".

In [ ]:
%%bash
# поднимаю контейнер в фоновом режиме
docker run -itd --rm --name c1 file_example
echo ""
# показываю что в конейнере нету файла
docker exec c1 ls
echo ""
# копирую файл через cp
docker cp my_datafile.csv c1:experimental/my_datafile.csv
# сомтрю в папку теперь
docker exec c1 ls
docker stop c1

0a4764eb41d1a3675983044244de0a3d4f891309f3c436413db479839b21abb5

experimental.py

experimental.py
my_datafile.csv
c1


### Контейнер -> хост

Копировать можно и в обратную сторону, при этом используется синтаксиc:

`docker cp <контейнер>:<путь к файлу в контейнере> <путь к файлу на компьютере>`

Так в следующем примере я из образа ubuntu достаю файл "/lib/os-release"

In [4]:
%%bash
docker run -itd --rm --name ubuntu ubuntu
docker cp ubuntu:/lib/os-release my-ubuntu-os
docker stop ubuntu
cat my-ubuntu-os

71f2be993a9c5435b509dc8b847e8712d8e12ea52963bdb99e604473b44fb7a0
ubuntu
PRETTY_NAME="Ubuntu 22.04.1 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.1 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


# Использование папки на хосте

Для того, чтобы подключить к контейнеру папку на хосте нужно использовать опцию `-v` коаманды `docker run`. Полный синтаксис следующий:

```
docker run \
    -v <папка на хосте1>:<папка в контейнере1> \
    -v <папка на хосте2>:<папка в контейнере2> \
    ...
```

При том под <папка на хостеi> может пониматься как:

- Путь к некоторой папке на хосте, такой способ подключения обычно называют **bind mount**, и используется он чаще для того чтобы передать в контейнер что-то характерное только для этого хоста;
- Название **volume**, такой способ более предпочтительный, и используется для сохранения информации которую "рождает" контейнер, по сути это таже папка на хосте только управляется docker-ом.

### Bind mount

В следующем примере я создаю папку `temp_folder`, её цепляю в контейнер под именем `temp_folder_inc_cont`. Из контейнера создаю в ней файл, выйдя их контейнера и даже удалив его, могу получить файл из папки хоста.

In [26]:
%%bash
mkdir temp_folder

docker run \
    -v $(pwd)/temp_folder:/temp_folder_in_cont \
    --rm -itd --name temp_example \
    ubuntu &> /dev/null
docker exec temp_example bash -c "echo \'привет из контейнера\' >> temp_folder_in_cont/hello"
docker stop temp_example &> /dev/null

cat temp_folder/hello
rm -r temp_folder

'привет из контейнера'


### Volume

Это файлы которые лежат на хосте и не зависят от конкретного образа, но зависят от docker. **Основное предназначение** - хранение данных.

##### **Служба `docker volume`**

`docker volume` - отдельная служба, которая имеет следующие комманды:

- `ls` - отобразить доступные volumes;
- `create` - создать volume;
- `rm` - удалить volume;
- `prune` - удалить valumes которые не используются в каком-либо контейнерe;
- `inspect` - позволяет получить информацию о volume (в том числе где он лежит на хосте).

##### **Пример**

Далее привен прмер использования volume, его содержание пошагово:
- Создается  `temp_volume`;
- Запускается контейнер ubuntu с именем `example_container`, к нему цепляется ранее созданный volume под папкой `temp_volume_cont`;
- В `temp_volume_cont` пишется файл;
- `example_container` останавливается и автоматически удаляется (так как был создан с опцией `--rm`);
- Далее через `inspect` показывается где можно найти volume на компьютере (поле `Mountpoint`), но туда из jupyter, к сожалению, не досучаться - нужны root права;
- `example_container` подниматеся, точно так-же как и раньше;
- И в папке, которой приделан volume находится, тот же файл, что был создан в предыдущей "реинканации контейнера".

In [ ]:
%%bash
docker volume create temp_volume &> /dev/null

docker run \
    -v temp_volume:/temp_volume_cont\
    --rm --name example_container -itd\
    ubuntu &> /dev/null
docker exec example_container bash -c "echo \'привет из volume\' >> temp_volume_cont/hello"
docker stop example_container &> /dev/null

docker volume inspect temp_volume

docker run \
    -v temp_volume:/temp_volume_cont\
    --rm --name example_container -itd\
    ubuntu &> /dev/null
docker exec example_container cat temp_volume_cont/hello
docker stop example_container &> /dev/null

docker volume rm temp_volume &> /dev/null

[
    {
        "CreatedAt": "2023-05-09T19:40:05+03:00",
        "Driver": "local",
        "Labels": null,
        "Mountpoint": "/var/lib/docker/volumes/temp_volume/_data",
        "Name": "temp_volume",
        "Options": null,
        "Scope": "local"
    }
]
'привет из volume'


### Тонкости


##### У контейнера root

Даже если для хоста файл или папка имеют особый уровень доступа, **контейнер всегда работает под root**. Таким образом если монтировать такую папку, это может привести к тому, что в ней начнутся несанкционированные изменения. Далее пример.

In [12]:
%%bash
# создаем папку и в ней файл и даже в него записываем
# сверхсекретное сообщение
mkdir secret_dir
touch secret_dir/secret_file
echo "super secret info" > secret_dir/secret_file
# закрываю доступ в папку
chmod 000 secret_dir


# убедимся, что через хост мы не может не получить инфу
# ни удалить её
echo "=====Через host====="
cat secret_dir/secret_file
rm secret_dir/secret_file


# поднимаем контейнер и монтируем в него данную папку
docker run --rm -itd --name perm_ex\
    -v $(pwd)/secret_dir:/experimental/secret_dir \
    ubuntu &> /dev/null
# и вауля, легко достаем "секретную информацию"
echo "=====Через docker===="
docker exec perm_ex cat experimental/secret_dir/secret_file
# или даже можем удалить файл
docker exec perm_ex rm experimental/secret_dir/secret_file


docker stop perm_ex &> /dev/null

=====Через host=====


cat: secret_dir/secret_file: Permission denied
rm: cannot remove 'secret_dir/secret_file': Permission denied


=====Через docker====
super secret info


##### Опция `ro` (read only)

Продолжая предыдущий подраздел, заметим, что при монтировании чего либо можно указать опцию `ro` (read only), которая запретит контейнеру изменять примонтированный файл.

In [14]:
%%bash
echo "some data" > ro_ex
# поднимаю контейнер с опцией ro
docker run --rm -idt --name ro_ex\
    -v $(pwd)/ro_ex:/experimental/ro_ex:ro\
    ubuntu &> /dev/null
# изменяю файл
docker exec ro_ex bash -c "echo \"new some data\" > ro_ex"
# вывожу новый файл
cat ro_ex
docker stop ro_ex &> /dev/null

some data


Так как бы я в примере выше не пытался из контейнера именить файл ro_ex - не получается.

### Вход с указанием пользователя

Так же проблему с тем что в контейнере постоянно root можно обойти, указав пользователя при запуске контейнера (опция `-u=<id>`). Так пример из подраздела `root контейнера` можно сделать следующим образом:

In [15]:
%%bash
# создаем папку и в ней файл и даже в него записываем
# сверхсекретное сообщение
mkdir secret_dir
touch secret_dir/secret_file
echo "super secret info" > secret_dir/secret_file
# закрываю доступ в папку
chmod 000 secret_dir

# поднимаем контейнер и монтируем в него данную папку
docker run --rm -itd --name perm_ex -u=1000\
    -v $(pwd)/secret_dir:/experimental/secret_dir \
    ubuntu &> /dev/null
echo "=====Попытка достучаться из контейнера====="
docker exec perm_ex cat secret_dir/secret_file

docker stop perm_ex &> /dev/null

=====Попытка достучаться из контейнера=====


cat: secret_dir/secret_file: No such file or directory


### .dockeringnore

Даже если мы монтируем файл который указан в .dockeringore мы все равно получим его в контейнере.

In [22]:
%%bash

echo "=====.dockerignore====="
cat .dockerignore
echo -e "\n=====ignore-file.txt====="
cat ignore_file.txt

# запускаем контейнер монтируя файл, который указан в .dockerignore
docker run --rm -itd --name ignore_ex\
    -v $(pwd)/ignore_file.txt:/app/ignore_file.txt\
    ubuntu &> /dev/null

echo "=====ignore-file из контейнера====="
# убеждаемся, что наш секретный файл преспокойно сидит в контейнере
docker exec ignore_ex cat app/ignore_file.txt

docker stop ignore_ex &> /dev/null

=====.dockerignore=====
ignore_file.txt
=====ignore-file.txt=====
file that should be ignored
=====ignore-file из контейнера=====
file that should be ignored


### volume по умолчанию

Некоторые контейнеры, когда поднимаются создают себе volume автоматически. Потому в один момент можно обнаружить, что у тебя весь диск забит. Например `yandex/clickhouse-server`. Так в следующем примере я поднимаю несколько контейнеров `yandex/clickhouse-server` и покзываю, что каждый из них создал по volume.

In [23]:
%%bash
# поднимаю clickhouse
docker run -d --name db_1 --rm yandex/clickhouse-server &> /dev/null
docker run -d --name db_2 --rm yandex/clickhouse-server &> /dev/null
docker run -d --name db_3 --rm yandex/clickhouse-server &> /dev/null

# смотрю список volume -
docker volume ls
docker stop db_1 db_2 db_3 &> /dev/null

DRIVER    VOLUME NAME
local     5e991d1877a4cf14eda2e93935789178f4918f3b11f9185d89b170ad38d88619
local     e5607f5376defe6b4318c46be41b315f8203104990c15a1669b9f98dcd9a296e
local     fd203d49fe52d15aacb0bb4788122d2f9becbb66a5c566817bb9c0478cc56901
